In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from config import Config as c

In [4]:
ham_dataset_dir = 'ham'
batch_size = 48
seed = 42

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
train_ds = keras.preprocessing.image_dataset_from_directory(ham_dataset_dir, validation_split=0.2, color_mode='rgb',
                                                            labels='inferred', shuffle=True, subset='training', image_size=(256, 256),
                                                            batch_size=batch_size, seed=seed)

val_ds = keras.preprocessing.image_dataset_from_directory(ham_dataset_dir, validation_split=0.2, color_mode='rgb',
                                                          labels='inferred', shuffle=True, subset='validation', image_size=(256, 256),
                                                          batch_size=batch_size, seed=seed)

Found 10015 files belonging to 7 classes.
Using 8012 files for training.
Found 10015 files belonging to 7 classes.
Using 2003 files for validation.


In [7]:
classes = train_ds.class_names

train_ds_single_batch = keras.preprocessing.image_dataset_from_directory(ham_dataset_dir, validation_split=0.2, color_mode='rgb',
                                                                         labels='inferred', shuffle=True, subset='training', image_size=(256, 256),
                                                                         batch_size=1, seed=seed)

Found 10015 files belonging to 7 classes.
Using 8012 files for training.


In [34]:
print(classes)

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


In [8]:
y = np.array([label.numpy()[0] for _, label in train_ds_single_batch])
class_weights_list = compute_class_weight('balanced', classes=np.unique(y), y=y)
print(class_weights_list)

[ 4.31913747  2.77135939  1.31107838 12.86035313  1.26611884  0.2138986
  9.78266178]


In [39]:
len_train = np.array([label.numpy()[0] for _, label in train_ds])
steps_per_epoch = len(len_train)//batch_size
print(steps_per_epoch)

3


In [38]:
validation_steps = len(val_ds)//batch_size
print(validation_steps)

0


In [9]:
class_weights = {}
for i in range(len(class_weights_list)):
    class_weights[i] = class_weights_list[i]

In [10]:
rescale = keras.layers.experimental.preprocessing.Rescaling(scale=1.0 / 255)

In [11]:
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds = val_ds.map(lambda x, y: (rescale(x), y))

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [12]:
"""
ResNet-18
Reference:
[1] K. He et al. Deep Residual Learning for Image Recognition. CVPR, 2016
[2] K. He, X. Zhang, S. Ren, and J. Sun. Delving deep into rectifiers:
Surpassing human-level performance on imagenet classification. In
ICCV, 2015.
"""


from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.models import Sequential
from keras.models import Model
import tensorflow as tf

In [13]:
class ResnetBlock(Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        
        self.bn_1 = BatchNormalization()
        self.conv_2 = Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = BatchNormalization()
        self.merge = Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = BatchNormalization()

    def call(self, inputs):
        res = inputs
        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out

In [14]:
class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = BatchNormalization()
        self.pool_2 = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.avg_pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out

In [44]:
model = ResNet18(7)
model.build(input_shape = (None,32,32,3))
#use categorical_crossentropy since the label is one-hot encoded
# from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 
model.summary()

Model: "res_net18_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_120 (Conv2D)         multiple                  9472      
                                                                 
 batch_normalization_120 (Ba  multiple                 256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_6 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 resnet_block_48 (ResnetBloc  multiple                 74368     
 k)                                                              
                                                                 
 resnet_block_49 (ResnetBloc  multiple                 74368     
 k)                                                    

In [45]:
from keras.callbacks import EarlyStopping

# es = EarlyStopping(patience= 8, restore_best_weights=True, monitor="val_acc")
#I did not use cross validation, so the validate performance is not accurate.
STEPS = len(y) // batch_size
# history = model.fit(train_ds, steps_per_epoch=STEPS, batch_size = batch_size, epochs=50, validation_data=val_ds,callbacks=[es])
history = model.fit(train_ds, steps_per_epoch=STEPS, batch_size = batch_size, epochs=50, validation_data=val_ds)

Epoch 1/50


ValueError: in user code:

    File "C:\Users\alif-\anaconda3\envs\tf2.5\lib\site-packages\keras\engine\training.py", line 946, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\alif-\anaconda3\envs\tf2.5\lib\site-packages\keras\engine\training.py", line 935, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\alif-\anaconda3\envs\tf2.5\lib\site-packages\keras\engine\training.py", line 928, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\alif-\anaconda3\envs\tf2.5\lib\site-packages\keras\engine\training.py", line 842, in train_step
        loss = self.compiled_loss(
    File "C:\Users\alif-\anaconda3\envs\tf2.5\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\alif-\anaconda3\envs\tf2.5\lib\site-packages\keras\losses.py", line 141, in __call__
        losses = call_fn

In [41]:
8012//batch_size

166

In [42]:
166*50

8300

In [43]:
# from keras.callbacks import EarlyStopping

# es = EarlyStopping(patience= 8, restore_best_weights=True, monitor="val_acc")
#I did not use cross validation, so the validate performance is not accurate.
# STEPS = len(y) // batch_size
# history = model.fit(train_ds, steps_per_epoch=STEPS, batch_size = batch_size, epochs=50, validation_data=val_ds,callbacks=[es])
history = model.fit(train_ds, 
                    steps_per_epoch= 8012//batch_size, 
                    epochs=20, 
                    validation_data=val_ds, 
                    validation_steps=2003//batch_size)

Epoch 1/20
166/166 [==============================] - 25s 150ms/step - loss: 0.0000e+00 - accuracy: 0.0517 - val_loss: 0.0000e+00 - val_accuracy: 0.0508
Epoch 2/20
166/166 [==============================] - 2s 14ms/step - loss: 0.0000e+00 - accuracy: 0.0227 - val_loss: 0.0000e+00 - val_accuracy: 0.0503
